## Structure mapping

In the example below, only structure elements  _S.a_ ,  _S.b_  and  _S.p_   of the  _S_  structure appear in `map` clauses of a `target` construct. Only these components have corresponding variables and storage on the device.   Hence, the large arrays,  _S.buffera_  and  _S.bufferb_ , and the  _S.x_  component have no storage  on the device and cannot be accessed.

Also, since the pointer member  _S.p_  is used in an array section of a  `map` clause, the array storage of the array section on the device,   _S.p[:N]_ , is **attached** to the pointer member  _S.p_  on the device. Explicitly mapping the pointer member  _S.p_  is optional in this case.

Note: The buffer arrays and the  _x_  variable have been grouped together, so that the components that will reside on the device are all together (without gaps). This allows the runtime to optimize the transfer and the storage footprint on the device.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name:       target_struct_map.1c
* type:       C
* version: omp_5.0
*/
#include <stdio.h>
#include <stdlib.h>
#define N 100
#define BAZILLION 2000000

struct foo {
  char buffera[BAZILLION];
  char bufferb[BAZILLION];
  float x;
  float a, b;
  float *p;
};

#pragma omp declare target
void saxpyfun(struct foo *S)
{
  int i;
  for(i=0; i<N; i++)
    S->p[i] = S->p[i]*S->a + S->b;
}
#pragma omp end declare target

int main()
{
  struct foo S;
  int i;

  S.a = 2.0;
  S.b = 4.0;
  S.p = (float *)malloc(sizeof(float)*N);
  for(i=0; i<N; i++) S.p[i] = i;

  #pragma omp target map(alloc:S.p) map(S.p[:N]) map(to:S.a, S.b)
  saxpyfun(&S);

  printf(" %4.0f %4.0f\n", S.p[0], S.p[N-1]);
        //     4  202  <- output
  return 0;
}



The following example is a slight modification of the above example for  a C++ class.  In the member function  _SAXPY::driver_   the array section  _p[:N]_  is **attached** to the pointer member  _p_  on the device.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name:       target_struct_map.2cpp
* type:       C++
* version: omp_5.0
*/
#include <cstdio>
#include <cstdlib>
#define N 100

class SAXPY {
  private:
   float a, b, *p;
  public:
   float buffer[N];

   SAXPY(float arg_a, float arg_b){ a=arg_a; b=arg_b; }
   void driver();
   void saxpyfun(float *p);
};

#pragma omp declare target
void SAXPY::saxpyfun(float *q)
{
  for(int i=0; i<N; i++)
    buffer[i] = q[i]*a + b;
}
#pragma omp end declare target

void SAXPY::driver()
{
  p = (float *) malloc(N*sizeof(float));
  for(int i=0; i<N; i++) p[i]=i;

  #pragma omp target map(alloc:p) map(to:p[:N]) map(to:a,b) \
              map(from:buffer[:N])   // attach(p) to device_malloc()
  {
    saxpyfun(p);
  }

  free(p);
}

int main()
{
  SAXPY my_saxpy(2.0,4.0);

  my_saxpy.driver();

  printf(" %4.0f %4.0f\n", my_saxpy.buffer[0], my_saxpy.buffer[N-1]);
         //   4   202     <- output

  return 0;
}

